<a href="https://colab.research.google.com/github/freha-mezzoudj/NLP_US_Airlines_tweets/blob/main/NaiveByaes_US_AirlinesSentiment_3Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **US Airlines Tweets Sentiment 3_Classes_Naive Bayes **

### **1. Problem Statement & approachs**

* Build a dictionary based on your training corpus. Calculate conditional probability of each token for each class (this is also called unigram probability). Then evaluate on test data and report accuracy.
* Try to improve your algorithm. Some suggestions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;i. Remove STOP words from the vocabulary that appear vary frequently but not related to the attitude or opinion of the writer. <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ii. Reduce the size of your vocabulary further by taking only top-k frequent word types that appear in the training dataset. Vary k and compare performance.

### **2. Importing Libraries & Loading Data**

In [ ]:
#importing the required libraries & packages
import pandas as pd
import numpy as np
import time
import argparse
import string
from sklearn.model_selection import train_test_split
from nltk.tokenize import regexp_tokenize
from datetime import datetime
import pytz

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/data")

In [ ]:
#loading the US Airline Sentiment data
data_frame = pd.read_csv('Tweets2.csv')
data_frame.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
data_frame.shape

(14640, 15)

### **3. Tweets Preprocessing**

The US Airline Sentiment dataset has so many features. Among them, in our project we will be working with the 'text' & 'airline_segment' features. Here, the 'text' is considered as a feature (X) and the 'airline_segment' as a target label (y) for the classifier.

The values in the target data are categorical and are 'neutral', 'positive' & 'negative'. For the easy computation, we are replacing the 'neutral', 'positive' & 'negative' with 0, 1 & 2 values respectively.

In [ ]:
#repalcing the categorical values of 'airline_sentiment' to numeric values
data_frame['airline_sentiment'].replace(('neutral', 'positive', 'negative'), (0, 1, 2), inplace=True)
data_frame['airline_sentiment'].value_counts()

2    9178
0    3099
1    2363
Name: airline_sentiment, dtype: int64

In [ ]:
#forming the feature & label variables
data = data_frame['text'].values.tolist()
labels = data_frame['airline_sentiment'].values.tolist()

In [ ]:
#First five samples text
data[:10]

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "@VirginAmerica and it's a really big bad thing about it",
 "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
 '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
 '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
 "@virginamerica Well, I didn't…but NOW I DO! :-D",
 "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me."]

In [ ]:
#first 5 samples label
labels[:10]

[0, 1, 0, 2, 2, 2, 1, 0, 1, 1]

### **4. Splitting the data for Classification**

The data splitting is done in 80-20 split using trian_test_split method of sklearn.

In [ ]:
#splitting the data into 80 and 20 split
train_X, test_X, y_train, y_test = train_test_split(data, labels, test_size=0.2, 
                                                    random_state=42, shuffle=True)

print(f'Number of training examples: {len(train_X)}')
print(f'Number of testing examples: {len(test_X)}')

Number of training examples: 11712
Number of testing examples: 2928


### **5. Tweets Preprocessing**

A process of transforming text into something an algorithm can digest is text processing. This includes:
* &nbsp;tokenizing the data
* &nbsp;removing the punctuation
* &nbsp;removing the stopwords
* &nbsp;stemming 
* &nbsp;lemmatization

As of now, we are only going to tokenize the data and work with it without removing the punctuation or stop words and apply any other text processing methods.

In [ ]:
# Here is a default pattern for tokenization
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """

In [ ]:
#funtion for tokenizing the data
""" Tokenize sentence with specific pattern
Arguments: text {str} -- sentence to be tokenized, such as "I love NLP"
Keyword Arguments: pattern {str} -- reg-expression pattern for tokenizer (default: {default_pattern})
Returns: list -- list of tokenized words, such as ['I', 'love', 'nlp'] """
def tokenize(text, pattern = default_pattern):

  text = text.lower()
  return regexp_tokenize(text, pattern)

In [ ]:
# Tokenize training text into tokens
tokenized_text = []
for i in range(0, len(train_X)):
    tokenized_text.append(tokenize(train_X[i]))

X_train = tokenized_text

# Tokenize testing text into tokens
tokenized_text = []
for i in range(0, len(test_X)):
    tokenized_text.append(tokenize(test_X[i]))

X_test = tokenized_text

In [ ]:
#tokenized train & test data
print(X_train[0], X_train[1])
print(X_test[0])

['@', 'united', 'you', 'are', 'offering', 'us', '8', 'rooms', 'for', '32', 'people', 'fail'] ['@', 'jetblue', 'jfk', 'nyc', 'staff', 'is', 'amazing', '.', 'the', 'lax', 'jetblue', '...', 'sending', 'an', 'email', 'with', 'details', 'but', 'it', 'was', 'a', 'disappointing', 'experience', '@', 'jetbluecheeps']
['@', 'southwestair', "you're", 'my', 'early', 'frontrunner', 'for', 'best', 'airline', 'oscars2016']


### **6. Building Dictionary**

Building dictionary of the training data.

In [ ]:
#building dictionary
def createDictionary(data):
  """ Function: To create a dictionary of tokens from the data
  Arguments: data in the type - list
  Returns: Sorted dictionary of the tokens and their count in the data """

  dictionary = dict()
  for sample in  data:
    for token in sample:
      dictionary[token] = dictionary.get(token, 0) + 1
  #sorting the dictionary based on the values
  sorted_dict = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
  return dict(sorted_dict)

In [ ]:
bag = createDictionary(X_train)
#top 10 items in the dictionary
print("Top 10 tokens in the training dictionary:\n")
list(bag.items())[:10]

Top 10 tokens in the training dictionary:



[('@', 13290),
 ('.', 12534),
 ('to', 6858),
 ('the', 4856),
 ('i', 4385),
 ('?', 3729),
 ('a', 3619),
 (',', 3354),
 ('united', 3338),
 ('you', 3284)]

### **7. Building the Navie Bayes Classifier**

Now, we define text classifier class called NBClassifier, which comprises of three functions:
* createDictionary()
* fit()
* predict()
* score()

**createDictionary():** This function takes in the tokenized text data and gives out the dictionary or the bag of words of the data.

**fit():** This function has all the word counts required to calculate the Navie Bayes Classifier probabilities and then fits the classifier on our training data.

**predict():** The test data is inputed to this function which determines the sentiment label based of each tweet by using the word counts computed during the training process (from fit function). In this step, Laplace smoothing is applied while computing Naïve Bayes probabilities for the test data.

**score():** Determine how many tweets are classified correctly and measures the performance of the model in terms of accuracy.

In [ ]:
#Navie Bayes Classifier 
class NBClassifier:

    def __init__(self, X_train, y_train, size):
      tz_NY = pytz.timezone('America/New_York') 
      print("Model Start Time:", datetime.now(tz_NY).strftime("%H:%M:%S"))
      self.X_train = X_train
      self.y_train = y_train
      self.size = size

    def createDictionary(self):
      """ Function: To create a dictionary of tokens from the data
      Arguments: data in the type - list
      Returns: Sorted dictionary of the tokens and their count in the data """
      dictionary = dict()
      for sample in  X_train:
        for token in sample:
          dictionary[token] = dictionary.get(token, 0) + 1
      #sorting the dictionary based on the values
      sorted_dict = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
      return dict(sorted_dict)
    
    def fit(self):
      """ Function: To compute the count of words in training data dictionary
        Arguments: Trianing data & Size of dictionary
        Returns: dictionary of tokens with their class wise probabilities """
      
      X_train_dict = self.createDictionary()
      if self.size == 'full':
        self.words_list = list(X_train_dict.keys())
        self.words_count = dict.fromkeys(self.words_list, None)
      else:
        self.words_list = list(X_train_dict.keys())[:int(self.size)]
        self.words_count = dict.fromkeys(self.words_list, None)
            
      #DataFrame of training data
      train = pd.DataFrame(columns = ['X_train', 'y_train'])
      train['X_train'] = X_train
      train['y_train'] = y_train

      train_0 = train.copy()[train['y_train'] == 0]
      train_1 = train.copy()[train['y_train'] == 1]
      train_2 = train.copy()[train['y_train'] == 2]

      #computing the prior of each class
      Pr0 = train_0.shape[0]/train.shape[0]
      Pr1 = train_1.shape[0]/train.shape[0]
      Pr2 = train_2.shape[0]/train.shape[0]
      
      self.Prior = np.array([Pr0, Pr1, Pr2])
        
      #converting list of lists into a list
      def flatList(listOfList):
        flatten = []
        for elem in listOfList:
          flatten.extend(elem)
        return flatten
  
      #Creating the data list for each class - tokens of each class
      X_train_0 = flatList(train[train['y_train'] == 0]['X_train'].tolist())
      X_train_1 = flatList(train[train['y_train'] == 1]['X_train'].tolist())
      X_train_2 = flatList(train[train['y_train'] == 2]['X_train'].tolist())
    
      self.X_train_len = np.array([len(X_train_0), len(X_train_1), len(X_train_2)])

      for token in self.words_list:
        #list to store three word counts of a token
        res = []

        #inserting count of token in class 0: Neutral
        res.insert(0, X_train_0.count(token))

        #inserting count of token in class 1: Positive
        res.insert(1, X_train_1.count(token))

          #inserting count of token in class 2: Negative
        res.insert(2, X_train_2.count(token))

        #assigning the count list to its token in the dictionary 
        self.words_count[token] = res
      return self

    def predict(self, X_test):
      """ Function: Predicts the label of the data
        Arguments: self and the test data
        Returns: List of predicted labels for the test data """     
      pred = []
      for sample in X_test:
        mul = np.array([1,1,1])
        for tokens in sample:
          vocab_count = len(self.words_list)
          if tokens in self.words_list:
            prob = ((np.array(self.words_count[tokens])+1) / (self.X_train_len + vocab_count))
          #except:
            #prob = ((np.array([0,0,0])+1) / (self.X_train_len + vocab_count))
          mul = mul * prob
        val = mul * self.Prior
        pred.append(np.argmax(val))
      tz_NY = pytz.timezone('America/New_York') 
      print("Model End Time:", datetime.now(tz_NY).strftime("%H:%M:%S"))
      return pred
    
    def score(self, pred, labels):
      """ Function: To compute the perfoemance of the model
        Arguments: self, predicted labels and actual labels of the test data
        Returns: Number of lables correctly predicted and the accuracy of the model """
      correct = (np.array(pred) == np.array(labels)).sum()
      accuracy = correct/len(pred)
      return correct, accuracy

### **8. Navie Bayes Classifier Training and Evaluation**

The Navie Bayes Classifier, NBClassifier takes three arguments:
* X_train: Features of training dataset
* y_train: Labels of training dataset
* size: Size of vacabulary to be used in the model

All three arguments are needed for the model to work.

In [ ]:
# Creating holders to store the model performance results
attributes = []
corr = []
acc = []

#function to call for storing the results
def storeResults(attr, cor,ac):
  attributes.append(attr)
  corr.append(round(cor, 3))
  acc.append(round(ac, 3))

In [ ]:
#training the classifier     
nb = NBClassifier(X_train, y_train, 'full')  
nb.fit()

#predicting the labels for test samples
y_pred = nb.predict(X_test)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred))

Model Start Time: 06:01:25
Model End Time: 06:02:17
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor1, acc1 = nb.score(y_pred, y_test)
print("Count of Correct Predictions:", cor1)
print("Accuracy of the model: %i / %i = %.4f " %(cor1, len(y_pred), acc1))

Count of Correct Predictions: 2291
Accuracy of the model: 2291 / 2928 = 0.7824 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Unprocessed Data', cor1, acc1)

The Navie Bayers Classifier that we trainined on the data predicts 78.24% of samples correctly. Now, to improve this number few more text processing methods are appilied on the training data and then the classifier is trained on this  modified data to predict the sentiment of the test samples.



### **9. Trying to improve the NBClassifier**

To improve the performance of the NBClassifier, 
* apply other text processing methods
* reduce the size of dictonary


#### **9.1. Further Processing Text Data**

In this step, we are going to apply two text processing methods on the previously tokenized data:
* remove the punctuation 
* remove stop words

##### **9.1.1. Remove Puntuation**

In [ ]:
#string of punctiations
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Removing the punctuation
'''Function: Removes the punctuation from the tokens
   Arguments: list of text data samples
   Returns: list of tokens of each sample without punctuation '''
def removePunctuation(data):
    update = []
    for sample in data:
        #removing punctuation from the tokens
        re_punct = [''.join(char for char in word if char not in string.punctuation) for word in sample]
        #removes the empty strings
        re_punct = [word for word in re_punct if word]
       
        update.append(re_punct)
    return update

In [ ]:
#Removing punctuation from training data text tokens  
X_train_P = removePunctuation(X_train)

#Removing punctuation from testing data text tokens
X_test_P = removePunctuation(X_test)

#train & test data after removing punctuation
print(X_train_P[0])
print(X_test_P[0])

['united', 'you', 'are', 'offering', 'us', '8', 'rooms', 'for', '32', 'people', 'fail']
['southwestair', 'youre', 'my', 'early', 'frontrunner', 'for', 'best', 'airline', 'oscars2016']


In [ ]:
#training the classifier     
nb_punct = NBClassifier(X_train_P, y_train, 'full')
nb_punct.fit()

#predicting the labels for test samples
y_pred_P = nb_punct.predict(X_test_P)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_P))

Model Start Time: 06:03:04
Model End Time: 06:03:56
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor2, acc2 = nb_punct.score(y_pred_P, y_test)
print("Count of Correct Predictions:", cor2)
print("Accuracy of the model: %i / %i = %.4f " %(cor2, len(y_pred_P), acc2))

Count of Correct Predictions: 2285
Accuracy of the model: 2285 / 2928 = 0.7804 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('No Punctuation Data', cor2, acc2)

##### **9.1.2. Remove Stopwords**

In [ ]:
'''Function: Removes the stopwords from the tokens
   Arguments: list of text data samples
   Returns: list of tokens of each sample without punctuation '''
def removeStopWords(data):
    update = []
    stopwords = ['the', 'at','i', 'of', 'us', 'have', 'a', 'you','ours', 'themselves', 
                 'that', 'this', 'be', 'is', 'for']
    for sample in data:
        #removing stopwords from tokenized data
        re_stop = [word for word in sample if word not in stopwords]
        
        update.append(re_stop)
    return update

In [ ]:
#Removing stopwords from training data text tokens  
X_train_S = removeStopWords(X_train)

#Removing stopwords from testing data text tokens
X_test_S = removeStopWords(X_test)

#train & test data after removing stopwords
print(X_train_S[0])
print(X_test_S[0])

['@', 'united', 'are', 'offering', '8', 'rooms', '32', 'people', 'fail']
['@', 'southwestair', "you're", 'my', 'early', 'frontrunner', 'best', 'airline', 'oscars2016']


In [ ]:
#training the classifier     
nb_stop = NBClassifier(X_train_S, y_train, 'full')
nb_stop.fit()

#predicting the labels for test samples
y_pred_S = nb_stop.predict(X_test_S)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_S))

Model Start Time: 06:05:16
Model End Time: 06:06:07
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor3, acc3 = nb_stop.score(y_pred_S, y_test)
print("Count of Correct Predictions:", cor3)
print("Accuracy of the model: %i / %i = %.4f " %(cor3, len(y_pred_S), acc3))

Count of Correct Predictions: 2300
Accuracy of the model: 2300 / 2928 = 0.7855 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Removed few Stopwords', cor3, acc3)

##### **9.1.3. Removing both Punctuation & Few Stopwords**

In [ ]:
#Removing stopwords from training data text tokens  
X_train_PS = removeStopWords(X_train_P)

#Removing stopwords from testing data text tokens
X_test_PS = removeStopWords(X_test_P)

#train & test data after removing stopwords
print(X_train_PS[0])
print(X_test_PS[0])

['united', 'are', 'offering', '8', 'rooms', '32', 'people', 'fail']
['southwestair', 'youre', 'my', 'early', 'frontrunner', 'best', 'airline', 'oscars2016']


In [ ]:
#training the classifier     
nb_PS = NBClassifier(X_train_PS, y_train, 'full')
nb_PS.fit()

#predicting the labels for test samples
y_pred_PS = nb_PS.predict(X_test_PS)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_PS))

Model Start Time: 06:08:21
Model End Time: 06:09:12
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor4, acc4 = nb_PS.score(y_pred_PS, y_test)
print("Count of Correct Predictions:", cor4)
print("Accuracy of the model: %i / %i = %.4f " %(cor4, len(y_pred_PS), acc4))

Count of Correct Predictions: 2283
Accuracy of the model: 2283 / 2928 = 0.7797 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Removed both Punctuation & Few Stopwords', cor4, acc4)

#### **9.2. Reducing the Dictionary Size**

To improve the model performance, we reduce the size of training dictionary further by taking only top-k frequent word types that appear in it. Here, we vary the value of k and compare the model performance.



In [ ]:
#total tokens in training dictionary
print('Total tokens in the dictionary:', len(bag))

Total tokens in the dictionary: 13606


##### **9.2.1. Considering Top 5k Tokens**

**5k Tokens of Vocabulary - Unprocessed data**

In [ ]:
#training the classifier - 5000 tokens 
nb_5k = NBClassifier(X_train, y_train, '5000')
nb_5k.fit()

#predicting the labels for test samples
y_pred_5k = nb_5k.predict(X_test)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_5k))

Model Start Time: 06:12:30
Model End Time: 06:12:49
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor5, acc5 = nb_5k.score(y_pred_5k, y_test)
print("Count of Correct Predictions:", cor5)
print("Accuracy of the model: %i / %i = %.4f " %(cor5, len(y_pred), acc5))

Count of Correct Predictions: 2332
Accuracy of the model: 2332 / 2928 = 0.7964 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('5k Tokens of Voab - Unprocessed Data', cor5, acc5)

**5k Tokens of Vocabulary - No Punctuation Data**

In [ ]:
#training the classifier - 5000 tokens 
nb_5k_P = NBClassifier(X_train_P, y_train, '5000')
nb_5k_P.fit()

#predicting the labels for test samples
y_pred_5k_P = nb_5k_P.predict(X_test_P)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_5k_P))

Model Start Time: 06:21:23
Model End Time: 06:21:42
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor6, acc6 = nb_5k.score(y_pred_5k_P, y_test)
print("Count of Correct Predictions:", cor6)
print("Accuracy of the model: %i / %i = %.4f " %(cor6, len(y_pred), acc6))

Count of Correct Predictions: 2309
Accuracy of the model: 2309 / 2928 = 0.7886 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('5k Tokens of Voab - No Punctuation Data', cor6, acc6)

**5k Tokens of Vocabulary - Removed few Stopwords**




In [ ]:
#training the classifier - 5000 tokens 
nb_5k_S = NBClassifier(X_train_S, y_train, '5000')
nb_5k_S.fit()

#predicting the labels for test samples
y_pred_5k_S = nb_5k_S.predict(X_test_S)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_5k_S))

Model Start Time: 06:24:42
Model End Time: 06:25:01
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor7, acc7 = nb_5k_S.score(y_pred_5k_S, y_test)
print("Count of Correct Predictions:", cor7)
print("Accuracy of the model: %i / %i = %.4f " %(cor7, len(y_pred), acc7))

Count of Correct Predictions: 2321
Accuracy of the model: 2321 / 2928 = 0.7927 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('5k Tokens of Voab - Removed few Stopwords', cor7, acc7)

**5k Tokens of Vocabulary - Removed both Punctuation & Few Stopwords**




In [ ]:
#training the classifier - 5000 tokens 
nb_5k_PS = NBClassifier(X_train_PS, y_train, '5000')
nb_5k_PS.fit()

#predicting the labels for test samples
y_pred_5k_PS = nb_5k_PS.predict(X_test_PS)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_5k_PS))

Model Start Time: 06:30:25
Model End Time: 06:30:44
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor8, acc8 = nb_5k_PS.score(y_pred_5k_PS, y_test)
print("Count of Correct Predictions:", cor8)
print("Accuracy of the model: %i / %i = %.4f " %(cor8, len(y_pred), acc8))

Count of Correct Predictions: 2296
Accuracy of the model: 2296 / 2928 = 0.7842 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('5k Tokens of Voab - Removed both Punctuation & Few Stopwords', cor8, acc8)

##### **9.2.2. Considering Top 10k Tokens**
**10k Tokens of Vocabulary - Unprocessed data**

In [ ]:
#training the classifier - 10000 tokens 
nb_10k = NBClassifier(X_train, y_train, '10000')
nb_10k.fit()

#predicting the labels for test samples
y_pred_10k = nb_10k.predict(X_test)

#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_10k))

Model Start Time: 06:32:46
Model End Time: 06:33:24
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor9, acc9 = nb_10k.score(y_pred_10k, y_test)
print("Count of Correct Predictions:", cor9)
print("Accuracy of the model: %i / %i = %.4f " %(cor9, len(y_pred), acc9))

Count of Correct Predictions: 2307
Accuracy of the model: 2307 / 2928 = 0.7879 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('10k Tokens of Voab - Unprocessed Data', cor9, acc9)

**10k Tokens of Vocabulary - No Punctuation Data**

In [ ]:
#training the classifier - 10000 tokens 
nb_10k_P = NBClassifier(X_train_P, y_train, '10000')
nb_10k_P.fit()

#predicting the labels for test samples
y_pred_10k_P = nb_10k_P.predict(X_test_P)
  
#Checking
print("NBClassifier Model miss any prediction???", len(X_test) != len(y_pred_10k_P))

Model Start Time: 06:35:25
Model End Time: 06:36:03
NBClassifier Model miss any prediction??? False


In [ ]:
#Performance of the classifier
cor10, acc10 = nb_10k_P.score(y_pred_10k_P, y_test)
print("Count of Correct Predictions:", cor10)
print("Accuracy of the model: %i / %i = %.4f " %(cor10, len(y_pred), acc10))

Count of Correct Predictions: 2287
Accuracy of the model: 2287 / 2928 = 0.7811 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('10k Tokens of Voab - No Punctuation Data', cor10, acc10)

**10k Tokens of Vocabulary - Removed few Stopwords**




In [ ]:
#training the classifier - 10000 tokens 
nb_10k_S = NBClassifier(X_train_S, y_train, '10000')
nb_10k_S.fit()

#Sredicting the labels for test samSles
y_pred_10k_S = nb_10k_S.predict(X_test_S)
  
#Checking
print("NBClassifier Model miss any Srediction???", len(X_test) != len(y_pred_10k_S))

Model Start Time: 06:37:32
Model End Time: 06:38:09
NBClassifier Model miss any Srediction??? False


In [ ]:
#Performance of the classifier
cor11, acc11 = nb_10k_S.score(y_pred_10k_S, y_test)
print("Count of Correct Predictions:", cor11)
print("Accuracy of the model: %i / %i = %.4f " %(cor11, len(y_pred), acc11))

Count of Correct Predictions: 2321
Accuracy of the model: 2321 / 2928 = 0.7927 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('10k Tokens of Voab - Removed few Stopwords', cor11, acc11)

**10k Tokens of Vocabulary - Removed both Punctuation & Few Stopwords**




In [ ]:
#training the claPSPSifier - 10000 tokenPS 
nb_10k_PS = NBClassifier(X_train_PS, y_train, '10000')
nb_10k_PS.fit()

#PSredicting the labelPS for tePSt PSamPSlePS
y_pred_10k_PS = nb_10k_PS.predict(X_test_PS)
  
#Checking
print("NBClaPSPSifier Model miSS any PSrediction???", len(X_test) != len(y_pred_10k_PS))

Model Start Time: 06:38:56
Model End Time: 06:39:34
NBClaPSPSifier Model miSS any PSrediction??? False


In [ ]:
#Performance of the classifier
cor12, acc12 = nb_10k_PS.score(y_pred_10k_PS, y_test)
print("Count of Correct Predictions:", cor12)
print("Accuracy of the model: %i / %i = %.4f " %(cor12, len(y_pred), acc12))

Count of Correct Predictions: 2293
Accuracy of the model: 2293 / 2928 = 0.7831 


In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('10k Tokens of Voab - Removed both Punctuation & Few Stopwords', cor12, acc12)

### **10. Comparing the Results**

In [ ]:
#creating dataframe
results = pd.DataFrame({ 'Data Modification': attributes,    
    'Correct Predictions': corr,
    'Model Accuracy': acc})

In [ ]:
results.sort_values(by=['Model Accuracy', 'Correct Predictions'], ascending=False)

,Data Modification,Correct Predictions,Model Accuracy
3,5k Tokens of Voab - Unprocessed Data,2332,0.796
5,5k Tokens of Voab - Removed few Stopwords,2321,0.793
9,10k Tokens of Voab - Removed few Stopwords,2321,0.793
4,5k Tokens of Voab - No Punctuation Data,2309,0.789
7,10k Tokens of Voab - Unprocessed Data,2307,0.788
6,5k Tokens of Voab - Removed both Punctuation &...,2296,0.784
10,10k Tokens of Voab - Removed both Punctuation ...,2293,0.783
0,Unprocessed Data,2291,0.782
8,10k Tokens of Voab - No Punctuation Data,2287,0.781
1,No Punctuation Data,2285,0.780


See you with another classifier :)